In [7]:
import boto3
import json
from decimal import Decimal

In [8]:
with open('20210513-180352_1085963612.json', 'r') as f:
    d = json.load(f, parse_float=Decimal)

In [22]:
list(list(d.values())[0].keys())

['estate_title',
 'estate_description_short',
 'estate_description_long',
 'estate_category_main_cb',
 'estate_disposition',
 'estate_rental_or_sell',
 'estate_locality_district',
 'estate_longitude',
 'estate_latitude',
 'estate_map_zoom',
 'estate_images',
 'seller_ico',
 'seller_email',
 'seller_numbers',
 'seller_web',
 'seller_address',
 'seller_name',
 'Cukrárna',
 'Kino',
 'Přírodní zajímavost',
 'Hřiště',
 'Večerka',
 'Hospoda',
 'Veterinář',
 'Bus MHD',
 'Pošta',
 'Lékárna',
 'Škola',
 'Restaurace',
 'Bankomat',
 'Lékař',
 'Vlak',
 'Obchod',
 'Školka',
 'Sportoviště',
 'Celková cena',
 'Poznámka k ceně',
 'ID zakázky',
 'Aktualizace',
 'Stavba',
 'Stav objektu',
 'Vlastnictví',
 'Podlaží',
 'Užitná plocha',
 'Plocha podlahová',
 'Balkón',
 'Topení',
 'Odpad',
 'Doprava',
 'Energetická náročnost budovy',
 'Bezbariérový',
 'Vybavení']

In [17]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('estates')

In [18]:
with table.batch_writer() as batch:
    for _id, items in d.items():
        batch.put_item(
            Item={
                'estate_id': _id,
                'items': items
            }
        )

In [ ]:
{  "Aktualizace" : { "S" : "Dnes" },  "Bankomat" : { "N" : "4812" },  "Bus MHD" : { "N" : "492" },  "Cena" : { "S" : "Informace o ceně na dotaz" },  "Cukrárna" : { "N" : "4641" },  "Doprava" : { "S" : "Silnice" },  "estate_category_main_cb" : { "N" : "3" },  "estate_description_long" : { "S" : "Nabízíme Vám ke koupi průmyslovou zónu, která se nachází v bezprostřední blízkosti dálnice D1, v obci Bravantice, nedaleko města Ostravy. Jedná se celkem o 9,4 ha pozemků, které jsou dostupné po obecní komunikaci. Dle platného územního plánu jsou pozemky určeny pro lehký průmysl, logistiku. Druhá zóna, která se nachází v blízkosti nabízených pozemků (velikost 30 ha) byla prodána v roce 2018. Více informací u makléře. Do konce května 2021 nabízíme prostřednictvím našeho partnera úrokové sazby od 2,09 % p.a. Více informací Vám sdělí náš specialista na prohlídce." },  "estate_description_short" : { "S" : "Pozemek ke komerční výstavbě 66 671 m² k prodeji Bravantice, okres Nový Jičín; 0 Kč za m²,  stavba." },  "estate_disposition" : { "N" : "18" },  "estate_images" : { "L" : [      { "S" : "https://d18-a.sdn.cz/d_18/c_img_QO_KT/fvipG.jpeg?fl=res,749,562,3|shr,,20|jpg,90" },      { "S" : "https://d18-a.sdn.cz/d_18/c_img_gV_Y/bIgFZc.jpeg?fl=res,749,562,3|shr,,20|jpg,90" },      { "S" : "https://d18-a.sdn.cz/d_18/c_img_QM_KD/znZpH.jpeg?fl=res,749,562,3|shr,,20|jpg,90" }    ]  },  "estate_latitude" : { "N" : "49.75010202246088" },  "estate_locality_district" : { "N" : "63" },  "estate_longitude" : { "N" : "18.089773111236717" },  "estate_map_zoom" : { "N" : "9" },  "estate_rental_or_sell" : { "N" : "1" },  "estate_title" : { "S" : "Prodej  komerčního pozemku 95 000 m²" },  "Hospoda" : { "N" : "195" },  "Hřiště" : { "N" : "2048" },  "ID zakázky" : { "S" : "112874" },  "Kino" : { "N" : "5025" },  "Komunikace" : { "S" : "Asfaltová" },  "Lékař" : { "N" : "3502" },  "Lékárna" : { "N" : "3905" },  "Plocha pozemku" : { "S" : "66671" },  "Pošta" : { "N" : "82" },  "Restaurace" : { "N" : "2685" },  "seller_address" : { "S" : "Bankovní 1826/6, 70200 Ostrava - Moravská Ostrava" },  "seller_email" : { "S" : "ostrava@rksting.cz" },  "seller_ico" : { "N" : "25842625" },  "seller_name" : { "S" : "Realitní kancelář STING, s.r.o." },  "seller_numbers" : { "L" : [      { "S" : "420596918261" }    ]  },  "seller_web" : { "S" : "http://www.rksting.cz" },  "Sportoviště" : { "N" : "4416" },  "Večerka" : { "N" : "82" },  "Vlak" : { "N" : "3856" },  "Škola" : { "N" : "104" },  "Školka" : { "N" : "173" }}